In [7]:
from rdflib import Graph, Namespace, RDF, Literal, URIRef
from pathlib import Path

# Konfiguration (Pfade anpassen!)
KG_INPUT_PATH = Path("D:/MA_Python_Agent/MSRGuard_Anpassung/KGs/Test_cleaned.ttl")
KG_OUTPUT_PATH = Path("D:/MA_Python_Agent/MSRGuard_Anpassung/KGs/Test_cleaned.ttl")

# Namespaces definieren (exakt wie in deinem kg_loader.py)
AG = Namespace('http://www.semanticweb.org/AgentProgramParams/')
DP = Namespace('http://www.semanticweb.org/AgentProgramParams/dp_')
OP = Namespace('http://www.semanticweb.org/AgentProgramParams/op_')

# Definition der IEC 61131-3 Standard-FBs
# Format: Name: {'inputs': {Name: Typ}, 'outputs': {Name: Typ}}
IEC_STANDARD_FBS = {
    # --- Bistable Elements ---
    "SR": { # Set Dominant
        "inputs": {"S1": "BOOL", "R": "BOOL"},
        "outputs": {"Q1": "BOOL"}
    },
    "RS": { # Reset Dominant
        "inputs": {"S": "BOOL", "R1": "BOOL"},
        "outputs": {"Q1": "BOOL"}
    },
    # --- Edge Detection ---
    "R_TRIG": { # Rising Edge
        "inputs": {"CLK": "BOOL"},
        "outputs": {"Q": "BOOL"}
    },
    "F_TRIG": { # Falling Edge
        "inputs": {"CLK": "BOOL"},
        "outputs": {"Q": "BOOL"}
    },
    # --- Counters ---
    "CTU": { # Count Up
        "inputs": {"CU": "BOOL", "R": "BOOL", "PV": "INT"},
        "outputs": {"Q": "BOOL", "CV": "INT"}
    },
    "CTD": { # Count Down
        "inputs": {"CD": "BOOL", "LD": "BOOL", "PV": "INT"},
        "outputs": {"Q": "BOOL", "CV": "INT"}
    },
    "CTUD": { # Count Up/Down
        "inputs": {"CU": "BOOL", "CD": "BOOL", "R": "BOOL", "LD": "BOOL", "PV": "INT"},
        "outputs": {"QU": "BOOL", "QD": "BOOL", "CV": "INT"}
    },
    # --- Timers ---
    "TP": { # Pulse
        "inputs": {"IN": "BOOL", "PT": "TIME"},
        "outputs": {"Q": "BOOL", "ET": "TIME"}
    },
    "TON": { # On-Delay
        "inputs": {"IN": "BOOL", "PT": "TIME"},
        "outputs": {"Q": "BOOL", "ET": "TIME"}
    },
    "TOF": { # Off-Delay
        "inputs": {"IN": "BOOL", "PT": "TIME"},
        "outputs": {"Q": "BOOL", "ET": "TIME"}
    }
}

def make_uri(name: str) -> URIRef:
    """Hilfsfunktion für URIs, analog zu deinem Loader."""
    safe = name.replace('.', '__dot__').replace(' ', '__leerz__')
    return URIRef(AG + safe)

def add_standard_fbs(graph: Graph):
    print("Füge IEC 61131-3 Standard-FBs hinzu...")
    
    for fb_name, ports in IEC_STANDARD_FBS.items():
        # 1. FB-Typ Instanz erstellen
        # URI Schema: StandardFBType_<Name>
        fb_uri = make_uri(f"StandardFBType_{fb_name}")
        
        graph.add((fb_uri, RDF.type, AG.class_StandardFBType))
        # Optional: Auch als FBType klassifizieren, falls Inference das nicht macht
        graph.add((fb_uri, RDF.type, AG.class_FBType))
        graph.add((fb_uri, DP.hasPOUName, Literal(fb_name)))
        graph.add((fb_uri, DP.hasPOUType, Literal("FunctionBlock")))
        graph.add((fb_uri, DP.hasPOULanguage, Literal("ST"))) # Standard als ST angenommen

        # 2. Ports erstellen
        all_ports = []
        # Inputs verarbeiten
        for port_name, port_type in ports["inputs"].items():
            all_ports.append((port_name, port_type, "Input"))
            
        # Outputs verarbeiten
        for port_name, port_type in ports["outputs"].items():
            all_ports.append((port_name, port_type, "Output"))
            
        # 3. Ports zum Graphen hinzufügen und verknüpfen
        for pname, ptype, pdir in all_ports:
            # URI Schema: Port_<FBName>_<PortName>
            port_uri = make_uri(f"Port_{fb_name}_{pname}")
            
            # Port Definition
            graph.add((port_uri, RDF.type, AG.class_Port))
            graph.add((port_uri, DP.hasPortName, Literal(pname)))
            graph.add((port_uri, DP.hasPortDirection, Literal(pdir)))
            graph.add((port_uri, DP.hasPortType, Literal(ptype)))
            
            # Verknüpfung: FB hat diesen Port
            graph.add((fb_uri, OP.hasPort, port_uri))
            
            print(f"  + {fb_name}.{pname} ({pdir}, {ptype})")


In [8]:

g = Graph()
print(f"Lade Graphen von {KG_INPUT_PATH}...")
g.parse(KG_INPUT_PATH, format="turtle")
    
# Standard FBs hinzufügen
add_standard_fbs(g)
    
# Speichern
print(f"Speichere erweiterten Graphen nach {KG_OUTPUT_PATH}...")
g.serialize(destination=KG_OUTPUT_PATH, format="turtle")
print("Fertig.")

Lade Graphen von D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\Test_cleaned.ttl...
Füge IEC 61131-3 Standard-FBs hinzu...
  + SR.S1 (Input, BOOL)
  + SR.R (Input, BOOL)
  + SR.Q1 (Output, BOOL)
  + RS.S (Input, BOOL)
  + RS.R1 (Input, BOOL)
  + RS.Q1 (Output, BOOL)
  + R_TRIG.CLK (Input, BOOL)
  + R_TRIG.Q (Output, BOOL)
  + F_TRIG.CLK (Input, BOOL)
  + F_TRIG.Q (Output, BOOL)
  + CTU.CU (Input, BOOL)
  + CTU.R (Input, BOOL)
  + CTU.PV (Input, INT)
  + CTU.Q (Output, BOOL)
  + CTU.CV (Output, INT)
  + CTD.CD (Input, BOOL)
  + CTD.LD (Input, BOOL)
  + CTD.PV (Input, INT)
  + CTD.Q (Output, BOOL)
  + CTD.CV (Output, INT)
  + CTUD.CU (Input, BOOL)
  + CTUD.CD (Input, BOOL)
  + CTUD.R (Input, BOOL)
  + CTUD.LD (Input, BOOL)
  + CTUD.PV (Input, INT)
  + CTUD.QU (Output, BOOL)
  + CTUD.QD (Output, BOOL)
  + CTUD.CV (Output, INT)
  + TP.IN (Input, BOOL)
  + TP.PT (Input, TIME)
  + TP.Q (Output, BOOL)
  + TP.ET (Output, TIME)
  + TON.IN (Input, BOOL)
  + TON.PT (Input, TIME)
  + TON.Q (Output, BOO